## following this [tutorial](https://www.youtube.com/playlist?list=PLxYC9c7VtjHT2uGV1_5zz6WgavvN_8Yql) by Antonio Longa

In [1]:
# !pip install torch-geometric
# !pip install numpy==1.23.5

In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid

/home/abir/miniconda3/envs/pygdebias_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/abir/miniconda3/envs/pygdebias_env/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import shutil
shutil.rmtree('dataset', ignore_errors=True)
dataset = Planetoid(root='dataset', name='Cora')

Processing...
Done!


In [4]:
print(dataset)
print(f"number of grpahs: {len(dataset)}")
print(f"number of classes: {dataset.num_classes}")
print(f"number of node features: {dataset.num_node_features}")
print(f"number of edge features: {dataset.num_edge_features}")

Cora()
number of grpahs: 1
number of classes: 7
number of node features: 1433
number of edge features: 0


In [6]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/home/abir/miniconda3/envs/pygdebias_env/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [5]:
data = dataset[0]

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = SAGEConv(
            dataset.num_node_features, 
            dataset.num_classes,
            aggr="mean"
        )
    
    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)
        

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

Net(
  (conv): SAGEConv(1433, 7, aggr=mean)
)


In [12]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits = model()
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        # logits is the matrix of prediction, max(1) is max values of each row
        # where [0] is the max value and [1] is the index of the max value
        # we need the index
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [13]:
best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch %10 == 0:
        print(log.format(epoch, train_acc, best_val_acc, test_acc))

Epoch: 010, Train: 1.0000, Val: 0.7020, Test: 0.6920
Epoch: 020, Train: 1.0000, Val: 0.7060, Test: 0.6970
Epoch: 030, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 040, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 050, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 060, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 070, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 080, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 090, Train: 1.0000, Val: 0.7100, Test: 0.6980
Epoch: 100, Train: 1.0000, Val: 0.7100, Test: 0.6980


In [ ]:
# logit_test = torch.tensor([[-1.8548, -2.0164, -2.1502, -1.2583, -2.1891, -2.4928, -2.1620],
#         [-1.8982, -2.3488, -2.5499, -2.5410, -0.8000, -2.5723, -2.6296],
#         [-2.1271, -2.2625, -2.4186, -2.0720, -0.7592, -3.1187, -3.0062],
#         [-1.3485, -2.2051, -2.3010, -1.9492, -1.9506, -2.1094, -2.0867],
#         [-1.9191, -2.0882, -2.0980, -1.2640, -2.3010, -2.0158, -2.3994],
#         [-2.2554, -1.8900, -1.1128, -2.3015, -2.0933, -2.2566, -2.4378],
#         [-0.6369, -2.6020, -2.8973, -2.3186, -2.3167, -3.0402, -2.3337],
#         [-1.9804, -1.9747, -2.0719, -1.5227, -2.1657, -1.9063, -2.1557],
#         [-2.0575, -2.2312, -1.9879, -1.0392, -2.5695, -2.2915, -2.3388],
#         [-2.0393, -2.0483, -1.4946, -2.2027, -2.2632, -1.8696, -1.9113],
#         [-1.2351, -1.7037, -2.3899, -2.2105, -2.0271, -2.4124, -2.2578],
#         [-1.0761, -2.2022, -2.1459, -2.1235, -2.1427, -2.5047, -2.1812]])
# logit_test.max(1)

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0])